# Analyzing Unicorn Companies

![Hand with calculator](calculator.png "Calculator")

## Background 

The objective of this analysis will be to support an investment firm in analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information will grant them with a competitive insight as to industry trends and how they should structure their portfolio looking forward.

## The Dataset

The information will be analyzed with the use of the `unicorns` database, which contains the following tables and fields:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


## Industries with the highest number of unicorn companies

In [5]:
-- Inspecting the top three industries which have had the highest number of unicorn companies (from years 2019-2021).
SELECT industry, COUNT(industries.company_id) as num_unicorns
FROM industries 
INNER JOIN dates
ON industries.company_id = dates.company_id
WHERE EXTRACT(year FROM date_joined) IN ('2019', '2020', '2021')
GROUP BY industry
ORDER BY num_unicorns DESC
LIMIT 3;

,industry,num_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


## Average valuations per year and industry

In [6]:
-- Previewing the average valuation per industry across the years. 
SELECT EXTRACT(year FROM date_joined) AS year, industry, COUNT(industries.company_id) AS num_unicorns, AVG(valuation) AS 		average_valuation
FROM industries 
INNER JOIN dates 
ON industries.company_id = dates.company_id
INNER JOIN funding 
ON dates.company_id = funding.company_id
GROUP BY year, industry
ORDER BY year DESC;

,year,industry,num_unicorns,average_valuation
0,2022,Artificial intelligence,10,1.600000e+09
1,2022,Auto & transportation,2,3.000000e+09
2,2022,Cybersecurity,8,1.000000e+09
3,2022,Data management & analytics,4,1.000000e+09
4,2022,E-commerce & direct-to-consumer,7,1.571429e+09
...,...,...,...,...
119,2012,E-commerce & direct-to-consumer,1,2.700000e+10
120,2012,Other,1,1.000000e+11
121,2011,Fintech,1,4.600000e+10
122,2011,Internet software & services,1,6.000000e+09


## Assessing the average valuation within top performing industries

In [7]:
-- Generating the final query to answer the business question. 
WITH top_performers AS 
	(SELECT industry, COUNT(industries.company_id) as num_unicorns
	FROM industries 
	INNER JOIN dates
	ON industries.company_id = dates.company_id
	WHERE EXTRACT(year FROM date_joined) IN ('2019', '2020', '2021')
	GROUP BY industry
	ORDER BY num_unicorns DESC
	LIMIT 3), 
yearly_rankings AS 
	(SELECT EXTRACT(year FROM date_joined) AS year, industry, COUNT(industries.company_id) AS num_unicorns, AVG(valuation) AS 		average_valuation
	FROM industries 
	INNER JOIN dates 
	ON industries.company_id = dates.company_id
	INNER JOIN funding 
	ON dates.company_id = funding.company_id
	GROUP BY year, industry
	ORDER BY year DESC)

SELECT 
	industry, 
	year, 
	SUM(num_unicorns) AS num_unicorns, 
	ROUND(AVG(average_valuation/1000000000),2) AS average_valuation_billions
FROM yearly_rankings
WHERE year IN ('2019', '2020', '2021') AND industry IN (SELECT industry FROM top_performers)
GROUP BY industry, year
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23


## Conclusion

During the years of 2019-2021, the three industries that have produced the higher numbers of unicorn companies include 1) E-commerce & direct-to-consumer 2) Fintech and 3) Internet software & services.

For these industries, the average valuation has ranged over the years. However, we can conclude that:
- E-commerce & direct-to-consumer: during the three years of observation the industry has created 75 unicorn companies, with an average valuation of $3.0 billion dollars. 
- Fintech: the industry has led to the creation of 173 unicorn companies, with a mean valuation of $4.6 billion dollars. 
- Internet software & services: during 2019-2021 there were 152 unicorn companies in the industry, with an average valuation of $3.5 billion dollars. 

From the top performing industries, Fintech has been the most outstanding one, as it has led to the highest number of unicorn companies created, as well as the one that has generated the highest average valuation. However, when investing, it might be a good idea to diversify the portfolio. This is why these three industries might be a good idea to focus on when the company structures their portfolio. 